In [1]:
import numpy as np
import pandas as pd
import geopandas
from geodatasets import get_path
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('Dataset .csv')

In [4]:
df.head(2)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591


In [5]:
df.isna().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [6]:
def getting_cat_cols(df):
    cat = []
    for name, dtype in df.dtypes.items():
        if dtype == 'object':
            cat.append(name)
    return cat

In [7]:
cat_cols = getting_cat_cols(df)

In [8]:
lr = LabelEncoder()
def label_encoding(df, cat_cols):
    for i in cat_cols:
        df[i] = lr.fit_transform(df[i])
    return df

In [11]:
# df = label_encoding(df, cat_cols)

In [9]:
df['Cuisines'] = df['Cuisines'].fillna('')

In [10]:
df['Cuisines']

0             French, Japanese, Desserts
1                               Japanese
2       Seafood, Asian, Filipino, Indian
3                        Japanese, Sushi
4                       Japanese, Korean
                      ...               
9546                             Turkish
9547     World Cuisine, Patisserie, Cafe
9548              Italian, World Cuisine
9549                     Restaurant Cafe
9550                                Cafe
Name: Cuisines, Length: 9551, dtype: object

In [11]:
df['Combined Features'] = (
    df['Cuisines'] + ' ' +
    df['Price range'].astype(str) + ' ' +
    df['Aggregate rating'].astype(str)
)


In [12]:
vectorizer = CountVectorizer()
feature_matrix = vectorizer.fit_transform(df['Combined Features'])

In [13]:
similarity_matrix = cosine_similarity(feature_matrix, feature_matrix)

In [14]:
user_preferences = {
    "cuisine": "Italian",
    "price_range": 2,
    "rating": 4.0
}

In [15]:
query = (
    user_preferences.get('cuisine', '') + ' ' +
    str(user_preferences.get('price_range', '')) + ' ' +
    str(user_preferences.get('rating', ''))
)

In [16]:
query_vector = vectorizer.transform([query])

In [17]:
query_similarity = cosine_similarity(query_vector, feature_matrix).flatten()

In [18]:
similar_indices = query_similarity.argsort()[-5:][::-1]

In [20]:
recommendations = df.iloc[similar_indices][['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating']]

In [21]:
print(recommendations)

                    Restaurant Name Cuisines  Price range  Aggregate rating
335                My Fathers Place  Italian            2               3.7
9414                      San Carlo  Italian            2               4.3
115        La Dolce Vita Ristorante  Italian            3               4.1
1019        The Dark Hour - Kitchen  Italian            1               0.0
3746  Diva - The Italian Restaurant  Italian            4               3.8
